In [ ]:
import re
import pandas as pd

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

In [ ]:
DEVELOPER_KEY='' # KEY 지울 것
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

youtube=build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

In [ ]:
search_response=youtube.search().list(
    q="인천 유나이티드",
    order='relevance',
    part='snippet',
    maxResults=1,
    ).execute()
# 검색 시 최상단에 인천 유나이티드 채널이 나옴

In [ ]:
channel_id=search_response['items'][0]['id']['channelId']

In [ ]:
# 인천 유나이티드 채널 ID
channel_id

'UCGA9gUrYCb4hRk_wHBzB_nQ'

In [ ]:
# 25번째 플레이리스트까지가 2019년~2023년이다.
playlists=youtube.playlists().list(
    channelId=channel_id,
    part='snippet',
    maxResults=25).execute()

In [ ]:
ids=[]
titles=[]

for i in playlists['items']:
    ids.append(i['id'])
    titles.append(i['snippet']['title'])
    
df=pd.DataFrame([ids,titles]).T
df.columns=['PlayLists','Titles']

In [ ]:
# 플레이리스트 ID를 가져오고
df_25 = df[:25]

In [ ]:
# 플레이리스트 내의 영상 ID는 50개까지 불러올 수 있다.
for i, j in enumerate(list(df_25['PlayLists'])):
    playlist_videos=youtube.playlistItems().list(
        playlistId=j,
        part='snippet',
        maxResults=50)

    playlistitems_list_response=playlist_videos.execute()
    nextPageToken = playlistitems_list_response.get('nextPageToken')

     
# nextPageToken으로 추가로 불러오기 가능해서 nextPageToken이 있는 경우 ID를 모두 합친다.
    while ('nextPageToken' in playlistitems_list_response):
        nextPage = youtube.playlistItems().list(
            part='snippet',
            playlistId=j,
            maxResults='50',
            pageToken=nextPageToken
            ).execute()

        playlistitems_list_response['items'] = playlistitems_list_response['items'] + nextPage['items']

        if 'nextPageToken' not in nextPage:
            playlistitems_list_response.pop('nextPageToken', None)
        else:
            nextPageToken = nextPage['nextPageToken']

    video_names=[]
    video_ids=[]
    date=[]

    for v in playlistitems_list_response['items']:
        video_names.append(v['snippet']['title'])
        video_ids.append(v['snippet']['resourceId']['videoId'])
        date.append(v['snippet']['publishedAt'])
        
    vdf=pd.DataFrame([date,video_names,video_ids]).T
    vdf.columns=['Date','Title','IDS']

# 플레이리스트 내 각 영상의 정보를 가져온다.
    category_id=[]
    views=[]
    likes=[]
    mins=[]
    seconds=[]
    title=[]
    date=[]

    for u in range(len(vdf)):
        request=youtube.videos().list(
        part='snippet,contentDetails,statistics',
        id=vdf['IDS'][u])
        
        response=request.execute()
        
        if response['items']==[]:
            ids.append('-')
            date.append('-')
            category_id.append('-')
            views.append('-')
            likes.append('-')
            
        else :
            title.append(response['items'][0]['snippet']['title'])
            date.append(response['items'][0]['snippet']['publishedAt'])
            category_id.append(response['items'][0]['snippet']['categoryId'])
            views.append(response['items'][0]['statistics']['viewCount'])
            likes.append(response['items'][0]['statistics']['likeCount'])
        
# 플레이리스트마다 vdatas 동적 변수에 저장한다.
    globals()['vdatas'+str(i)]=pd.DataFrame([title,date,category_id,views,likes]).T
    globals()['vdatas'+str(i)].columns=['title','date','category_id','views','likes']

In [ ]:
# 모든 플레이리스트를 합친다.
vdatas = pd.concat([vdatas0,vdatas1,vdatas2,vdatas3,vdatas4,vdatas5,vdatas6,vdatas7,vdatas8,vdatas9,
                    vdatas10,vdatas11,vdatas12,vdatas13,vdatas14,vdatas15,vdatas16,vdatas17,vdatas18,vdatas19,
                    vdatas20,vdatas21,vdatas22,vdatas23,vdatas24])

In [ ]:
vdatas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 0 to 37
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        698 non-null    object
 1   date         698 non-null    object
 2   category_id  698 non-null    object
 3   views        698 non-null    object
 4   likes        698 non-null    object
dtypes: object(5)
memory usage: 32.7+ KB


In [ ]:
vdatas.head()

,title,date,category_id,views,likes
0,"""결과를 만들어 낸 선수들에게 칭찬해 주고 싶습니다"" | 공식 기자회견 10월 11...",2022-10-14T10:00:15Z,17,883,62
1,"""승점 1점에 만족하지 않고 잘 준비하고 대비하겠습니다"" | 공식 기자회견 9월 1...",2022-09-17T06:00:09Z,17,734,58
2,"""시즌 첫 3연승 할 좋은 기회가 왔다"" | 공식 기자회견 8월 13일 전북전",2022-08-17T10:35:53Z,17,1466,95
3,"""선수들과 약속한 걸 지킬 수 있어 기분 좋았습니다"" | 공식 기자회견 5월 29일...",2022-06-01T08:00:04Z,17,1835,51
4,"""본인의 가치를 더 증명했으면 좋겠습니다"" | 공식 기자회견 5월 17일 대구FC전",2022-05-20T09:00:12Z,17,2331,53


In [ ]:
vdatas['category_id'].value_counts()

17    521
24    175
20      2
Name: category_id, dtype: int64

In [ ]:
# 연월일만 남기고 시간 순 정리
vdatas['date'] = vdatas['date'].str.split('T').str[0]
vdatas['date'] = pd.to_datetime(vdatas['date'])
vdatas = vdatas.sort_values('date')
# 컬럼 타입 변경
vdatas['views'] = vdatas['views'].astype(int)
vdatas['likes'] = vdatas['likes'].astype(int)
# 정리
# vdatas = vdatas.drop('category_id', axis=1)
vdatas = vdatas.reset_index(drop=True)

In [ ]:
# vdatas.to_csv('new_incheon_youtube.csv', index=False)



---



In [ ]:
c1 = vdatas[vdatas['date'].dt.year == 2022]

In [ ]:
c2 = c1[c1['date'] <= '2022-10-16']

In [ ]:
c2.head()

,title,date,category_id,views,likes
563,[IUFC TV] '임대 종료' 민성준 인천 복귀 현장 | 민성준 오피셜 인터뷰,2022-01-01,17,1839,77
564,[IUFC TV] ‘헌신의 아이콘’ 베테랑 미드필더 여름 영입 | 여름 오피셜 인터뷰,2022-01-02,17,4190,142
565,[IUFC TV] 국가대표 출신 수비수 이주용 오피셜 현장 | 이주용 오피셜 인터뷰,2022-01-03,17,3250,112
566,[IUFC TV] ‘떠오르는 신성’ 홍시후 오피셜 현장 | 홍시후 오피셜 인터뷰,2022-01-05,17,3853,126
567,[IUFC TV] ‘팔방미인’ 미드필더 이동수 오피셜 현장 | 이동수 오피셜 인터뷰,2022-01-06,17,2342,96
